## In-progress

In [16]:
%load_ext autoreload
%autoreload 2

In [15]:
import os
os.getcwd()

'/Users/pradip.gupta/personal-projects/charni/notebooks'

In [17]:
import sys, glob, shutil
os.chdir(os.path.dirname(os.getcwd()))
os.getcwd()

'/Users/pradip.gupta/personal-projects/charni'

## Squeezenet

In [1]:
from keras import backend as K
from keras.layers import Input, Convolution2D, MaxPooling2D, Activation, concatenate, Dropout, warnings
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.models import Model
from keras.utils import get_file
from keras.utils import layer_utils

Using TensorFlow backend.


In [2]:
sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"

WEIGHTS_PATH = "https://github.com/rcmalli/keras-squeezenet/releases/download/v1.0/squeezenet_weights_tf_dim_ordering_tf_kernels.h5"
WEIGHTS_PATH_NO_TOP = "https://github.com/rcmalli/keras-squeezenet/releases/download/v1.0/squeezenet_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [3]:
# Modular function for Fire Nodes
def fire_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 3
    
    x = Convolution2D(squeeze, (1, 1), padding='valid', activation='relu', name=s_id + sq1x1)(x)

    left = Convolution2D(expand, (1, 1), padding='valid', activation='relu', name=s_id + exp1x1)(x)

    right = Convolution2D(expand, (3, 3), padding='same', activation='relu', name=s_id + exp3x3)(x)

    x = concatenate([left, right], axis=channel_axis, name=s_id + 'concat')
    
    return x

In [29]:
input_shape=(64, 64, 3)
include_top = False
pooling = "avg"

In [21]:
img_input = Input(shape=input_shape)

x = Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(img_input)
x = Activation('relu', name='relu_conv1')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

In [22]:
x = fire_module(x, fire_id=2, squeeze=16, expand=64)
x = fire_module(x, fire_id=3, squeeze=16, expand=64)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

In [23]:
x = fire_module(x, fire_id=4, squeeze=32, expand=128)
x = fire_module(x, fire_id=5, squeeze=32, expand=128)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

In [24]:
x = fire_module(x, fire_id=6, squeeze=48, expand=192)
x = fire_module(x, fire_id=7, squeeze=48, expand=192)
x = fire_module(x, fire_id=8, squeeze=64, expand=256)
x = fire_module(x, fire_id=9, squeeze=64, expand=256)

In [25]:
if include_top:
    # It's not obvious where to cut the network... 
    # Could do the 8th or 9th layer... some work recommends cutting earlier layers.

    x = Dropout(0.5, name='drop9')(x)

    x = Convolution2D(classes, (1, 1), padding='valid', name='conv10')(x)
    x = Activation('relu', name='relu_conv10')(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation('softmax', name='loss')(x)
else:
    if pooling == 'avg':
        x = GlobalAveragePooling2D()(x)
    elif pooling=='max':
        x = GlobalMaxPooling2D()(x)
    elif pooling==None:
        pass
    else:
        raise ValueError("Unknown argument for 'pooling'=" + pooling)

In [26]:
model = Model(img_input, x, name='squeezenet')

In [27]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 31, 31, 64)   1792        input_2[0][0]                    
__________________________________________________________________________________________________
relu_conv1 (Activation)         (None, 31, 31, 64)   0           conv1[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 15, 15, 64)   0           relu_conv1[0][0]                 
__________________________________________________________________________________________________
fire2/sque

In [28]:
model.load_weights("data/pretrained/squeezenet_weights_tf_dim_ordering_tf_kernels_notop.h5", reshape=True)